## CallBack

CallBack功能，允许用户在**求解过程中**，获取信息、终止优化、加入额外约束条件、加入自己开发的算法等

且Callback函数使用时需要注意两个重要参数:
1. where: 回调函数的触发节点
2. what: 获取何种信息, what能获取什么取决于当前CallBack的where。(比如在轮询回调POLLING阶段，什么信息都无法获得)

定义CallBack函数: def 函数名(model, where):...

调用CallBack函数: m.optimize(CallBack函数名)

以下为简单的可用where，可用的what太多，暂不列出; 具体参见refman.pdf - page817

|where|数值|优化器状态|
|:-|:-|:-|
|POLLING|0|轮询回调|
|PRESOLVE|1|预处理|
|SIMPLEX|2|单纯形|
|MIP|3|当前MIP|
|MIPSOL|4|发现新的MIP解|
|MIPNODE|5|当前探索节点|
|MESSAGE|6|打印出Log信息|
|BARRIER|7|当前内点法|
|MULTIOBJ|8|当前多目标|

### CallBack + RINS heuristic Example

**最大割问题**

   
  假设图中线段都被赋予了权重，希望找到一个方案将顶点分成两个子集(即为m,n)，使得属于不同子集点的连线权重和最大。例如图中的方案m={A,E,C}; n={B,D}，权重总和=AB+BE+BC+CD+ED

<img src="class3_pic1.jpg" style="zoom:50%">

- 参数
    - $C_{ij}$ 为 线段ij的权重，N顶点数量
- 变量
    - $x_i = 1$, if $i \in m$ or $x_i = -1$, if $i \in n$
    - 即当$x_i=1$时，顶点i属于集合m
    - 即当$x_i=-1$时，顶点i属于集合n
- 目标函数
    - max{$\frac{1}{4} \sum_{i=1}^{N} \sum_{j=1}^{N} C_{ij}(1 - x_i x_j)$}
    - 上式的$\frac{1}{4}$是因为当ij为不同集合时，$(1 - x_i x_j)$系数为2; 并且ij和ji都算了一次，所以需要乘上$\frac{1}{4}$使得其合理
- 约束
    - $x_i \in \{-1, 1\}$
    - 因为Gurobi没有[-1,1]这样的变量，所以需要做一个处理
        - $x = 2y - 1$ 且 y为binary

In [106]:
# 不使用CallBack + RINS heuristic而直接求解

# Import Env
import random
import gurobipy as gp
from gurobipy import GRB

# Generate Cmatrix
N = 10
random.seed(1)
Cmatrix = {(i,j):random.randint(0,100) for i in range(N) for j in range(N)}

# Create Model
m = gp.Model('class2_example1-1')

# Create Vars
y = m.addVars(N, vtype=GRB.BINARY, name='y')
x = m.addVars(N, lb=-1.0, ub=1.0, vtype=GRB.INTEGER, name='x')

# Create Objective
m.setObjective(0.25 * sum(Cmatrix[i,j] * (1 - x[i]*x[j]) for i in range(N) for j in range(N)), 
               GRB.MAXIMIZE)

# Create Constraint
m.addConstrs(
    (x[i] == 2 * y[i] - 1 for i in range(N)),
    name='C0'
)

# optimize
m.optimize()

# write into lp
m.write('class3_example1-1.lp')

# Print result
for v in m.getVars():
    print('{} {}'.format(v.varName, v.x))
print('Obj: {}'.format(m.objVal))

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 10 rows, 20 columns and 20 nonzeros
Model fingerprint: 0x6c29a9f9
Model has 54 quadratic objective terms
Variable types: 0 continuous, 20 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 10 rows and 10 columns
Presolve time: 0.00s
Presolved: 45 rows, 55 columns, 135 nonzeros
Variable types: 0 continuous, 55 integer (55 binary)

Root relaxation: objective -2.541500e+03, 17 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2541.50000    0   10   -0.00000 2541.50000      -     -    0s
H    0     0                    1463.5000000 2541.50000  73.7%     

In [ ]:
gp.abs_()

**RINS heuristic核心思想:**
- 随着整数规划模型的求解进程，节点松弛模型的解与最优解之间的差距可能会越来越小，体现在松弛解的部分变量值与最优解对应变量值相等或差距很小。因此利用松弛模型的信息可能会更快发现高质量的可行解
- RINS Heuristic基于上面的思想，在求解过程中抓取节点松弛解(可能是部分整数，部分小数)， 固定模型中对应的变量的取值构造一个子模型(规模往往远小于原模型)，然后求解子模型。如果发现了更好的可行解，把解传递给优化器让其在它的基础上继续求解原模型

In [104]:
# 使用RINS heuristic求解

from gurobipy import *
import random

def RINScallback(model, where):
    if where == GRB.Callback.MIPNODE:
        # MIP node callback
        if model.cbGet(GRB.Callback.MIPNODE_NODCNT) % 100 == 0 and \
           model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.OPTIMAL:
            submodel = model.copy()
            suby = submodel.getVars()
            #获得节点松弛解
            yrelaxation = model.cbGetNodeRel(model._y)
            #固定变量取值
            for i in range(model._N):
                if abs(yrelaxation[i])<0.01:
                    suby[i].ub = 0
                elif abs(yrelaxation[i]-1)<0.01:
                    suby[i].lb = 1
            submodel.setParam(GRB.Param.TimeLimit, 30)
            submodel.optimize()
            if submodel.objval > model.cbGet(GRB.Callback.MIPNODE_OBJBST):
                #将解传递给原模型
                for i in range(model._N):
                    if abs(suby[i].x)<0.001:
                        model.cbSetSolution(model._y[i], 0.0)
                    elif abs(suby[i].x-1)<0.001: 
                        model.cbSetSolution(model._y[i], 1.0)

try:
    random.seed(1)
    N = 10
    #随机产生线段权重
    random.seed(1)
    Cmatrix = {(i,j):random.randint(0,100) for i in range(N) for j in range(N)}
    m = Model('MaximumCut')
    #添加变量
    y = m.addVars(N, vtype=GRB.BINARY, name='y')
    #构造目标函数
    obj = QuadExpr()
    for i in range(N):
        for j in range(N):
            obj = obj+Cmatrix[i,j]*(y[i]+y[j]-2*y[i]*y[j])
    m.setObjective(0.5*obj, -1)
    #设置求解时间
    m.Params.TimeLimit = 600
    #外部变量
    m._y = y
    m._N = N
    #求解
    m.optimize(RINScallback)
    #获得目标值和变量值
    print("Obj = ",m.ObjVal)
    for i in range(N):
        print(y[i].VarName,' = ',y[i].x)
    
except GurobiError:
    print('Error reported')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 0 rows, 10 columns and 0 nonzeros
Model fingerprint: 0x7341e596
Model has 54 quadratic objective terms
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [4e+02, 6e+02]
  QObjective range [6e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 45 rows, 55 columns, 135 nonzeros
Variable types: 0 continuous, 55 integer (55 binary)

Root relaxation: objective -2.541500e+03, 17 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2541.50000    0   10   -0.00000 2541.50000      -     -    0s
H    0     0    

## 常用线性方法

### 广义约束 - Max

$z = max(x, y, 3)$
- 在Gurobi中，可以直接写成 m.addConstr(z == max_(x, y, 3))
- 但是如果不用广义约束，则我们需要将其转化成其他线性条件
    - 因为z为[x, y, 3]中的最大值, 则可以写出
        - $ x \le z, y \le z, 3 \le z$
    - 上述式子仅确认了z的上界，我们还需要确认z的下界, 即$x \ge z, y \ge z, 3 \ge z$ 至少有一个约束成立, 则可以写成下述形式
        - $x \ge z - M(1 - u_1)$
        - $y \ge z - M(1 - u_2)$
        - $3 \ge z - M(1 - u_3)$
        - $u_1 + u_2 + u_3 \ge 1$
        - $u_1, u_2, u_3 \in \{0,1\}$
        - 即$u_i = 1$时，对应的那条限制成立；而至少有一个$u_i = 1$, 则至少有一个约束成立
        - $M$为一个无穷大的常数
- 将上述6条式子联立，则为线性化Max的方法

### 广义约束 - Min

$z = min(x, y, 3)$
- 在Gurobi中，可以直接写成 m.addConstr(z == min_(x, y, 3))
- 但是如果不用广义约束，则我们需要将其转化成其他线性条件
    - 因为z为[x, y, 3]中的最小值, 则可以写出
        - $ x \ge z, y \ge z, 3 \ge z$
    - 上述式子仅确认了z的上界，我们还需要确认z的下界, 即$x \le z, y \le z, 3 \le z$ 至少有一个约束成立, 则可以写成下述形式
        - $x \le z - M(1 - u_1)$
        - $y \le z - M(1 - u_2)$
        - $3 \le z - M(1 - u_3)$
        - $u_1 + u_2 + u_3 \ge 1$
        - $u_1, u_2, u_3 \in \{0,1\}$
        - $M$为一个无穷大的常数
        - 即$u_i = 1$时，对应的那条限制成立；而至少有一个$u_i = 1$, 则至少有一个约束成立
- 将上述6条式子联立，则为线性化Min的方法

### 目标函数中存在绝对值

$min \sum_{i} c_i |x_i|$

$Ax = b$

$x_i$ free, $c_i \ge 0$, $\forall i$

- 在Gurobi中，可以将其转化为 m.addConstr(y==abs_(x)), 然后将目标函数中的$|x_i|$替代为$y_i$
- 线性化方法1:
    - $y_i = |x_i|$, 可以推出下式:
        - $y_i \ge x_i$, $y_i \ge -x_i$
        - 但该式只确定了$y_i$的下界，$y_i$可以取到无穷大
        - 但是如下述规划所示，我们要求的是min，所以会使得$y_i$尽量的小，所以没必要确定$y_i$的上界
    - 则可以获得如下规划
        - $min \sum_{i} c_i y_i$
        - $Ax = b$
        - $y_i \ge x_i$, $y_i \ge x_i$
        - $x_i$ free, $c_i \gt 0$

- 线性化方法2:
    - 定理: $\forall \ x, \exists \ u,v \ge 0$, 使得$x = u - v$, $|x| = u + v$, 其中 $u = \frac{|x|+x}{2}$, $v = \frac{|x|-x}{2}$
    - 根据如上定理，可以将其规划转化为
        - $min \sum_i c_i (u_i + v_i)$
        - $A(u - v) = b$
        - $u,v \ge 0$

### Maxmin / Minmax 目标函数


- $max(min_{k \in k}(\sum_i c_{ki} x_i))$
    - 可以线性化成:
        - $max (z)$
        - $z \le \sum_i c_{ki} x_i$, $\forall \ k \in K$


- $min(max_{k \in k}(\sum_i c_{ki} x_i))$
    - 可以线性化成:
        - $min (z)$
        - $z \ge \sum_i c_{ki} x_i$, $\forall \ k \in K$

In [107]:
# 例子

# max(min(x+2y+10, 3x+y+1))
# ->
# max z
# z <= x + 2y + 10
# z <= 3x + y + 1

### Maxmax / Minmin 目标函数

- $max(max_{k \ \in k} \sum_i c_{ki} x_i)$
    - 可以线性化成(结合了2.1与2.4):
        - $max (z)$
        - $\sum_i c_{ki} x_i \ge z - M(1 - y_k)$, $\forall \ k \in K$
        - $\sum_k y_k \ge 1$
        - $y_k \ \in \{0,1\}$
        
- $min(min_{k \ \in k} \sum_i c_{ki} x_i)$
    - 可以线性化成(结合了2.1与2.4):
        - $min (z)$
        - $\sum_i c_{ki} x_i \le z - M(1 - y_k)$, $\forall \ k \in K$
        - $\sum_k y_k \ge 1$
        - $y_k \ \in \{0,1\}$
    - $M$为一个无穷大的常数

In [109]:
# 例子

# max(max(x+2y+10, 3x+y+1))
# ->
# max(z)
# x + 2y + 10 >= z - M(1 - u)
# 3x + y + 1 >= z - M(1 - v)
# u + v >= 1
# u,v binary

# x + 2y + 10 <= z 可被省略, 因其优化方向为最大值
# 3x + y + 1 <= z 可被省略, 因其优化方向为最大值

### 带fixed cost的目标函数

- $min \ f(x) = \begin{cases} 
0,  & \mbox{if }x=0 \\
cx + k, & \mbox{if }x \gt 0 , k \gt0
\end{cases}$
    - 可以线性化成 (即引入了一个二分类的y，当y为0时表示x为0，当y为1时表示x大于0)
        - $min (cx + ky)$
        - $ x \le My$
        - $ y \in \{0,1\}$
        - $M$为一个无穷大的常数

### 分式目标函数

- $min \sum_i(c_i x_i + \alpha) / \sum_i(d_i x_i + \beta)$
- $\sum_i a_{ij} x_i \le b_j$, $\forall j \in J$
- $\sum_i d_i x_i + \beta \gt 0$, $x_i \ge 0$, $\forall j \in J$

    - 线性化方法: 令 $y = \frac{1}{\sum_i(d_ix_i + \beta)} \gt 0$
        - $min \sum_i (c_i x_i y + \alpha y)$
        - $\sum_i a_{ij} x_i \le b_j$, $\forall j \in j$
        - $\sum_i d_i x_i y + \beta y = 1$, $\forall j \in J$
        - $y \gt 0$, $x_i \ge 0$, $forall j \in J$
    - 由上述式子含两个变量相乘，为二次方，所以需要进一步线性化。加上 $z_i = x_i y$, 可得
        - $min \sum_i (c_i z_i + \alpha y)$
        - $\sum_i a_{ij} z_i \le b_j y$, $\forall j \in j$
        - $\sum_i d_i z_i + \beta y = 1$, $\forall j \in J$
        - $y \gt 0$, $z_i \ge 0$, $forall j \in J$        
    

In [111]:
# 例子
# min (2x + y + 1) / (x + 3y)
# 5x +y <= 6
# x + 3y > 0, x,y >= 0
# ->
# 令: z = 1 / (x + 3y) > 0
# min (2xz + yz + z)
# 5x + y <= 6
# xz + 3yz = 1
# x,y >= 0, z > 0
# ->
# 令 u = xz, v = yz, 5xz+yz <= 6z
# min (2u + v + z)
# 5u + v <= 6z
# u + 3v = 1
# z>0
# u,v >= 0

### 逻辑或 约束

- $\sum_j a_{1j} x \le b_1$ or $\sum_j a_{2j} x \le b_2$ (两个约束至少一个成立)
    - 线性化方法:
        - $\sum_j a_{ij} x \le b_i + M(1 - y_i)$, $i = 1,2$
        - $\sum_i y_i \ge 1$
        - $y \in \{0,1\}$, $i = 1,2$
        - $M$为一个无穷大的常数

In [ ]:
# 例子
# x + 2y + 10 <= 15 or 3x + y + 1 <= 5
# ->
# x + 2y + 10 <= 15 + M(1 - u)
# 3x + y + 1 <= 5 + M(1 - v)
# u + v >= 1
# u,v in {0,1}

- $\sum_j a_{1j} x \le b_1$ or $\sum_j a_{2j} x = b_2$ (两个约束至少一个成立)
    - 线性化方法:
        - $\sum_j a_{ij} x \le b_i + M(1 - y_i)$, $i = 1,2$
        - $\sum_j a_{2j} x \ge b_2 - M(1 - y_2)$
        - $\sum_i y_i \ge 1$
        - $y_i \in \{0,1\}$, $i = 1,2$

In [130]:
# 例子
# x + 2y + 10 <= 15 or 3x + y + 1 = 5
# ->
# x + 2y + 10 <= 15 + M(1 - u)
# 3x + y + 1 <= 5 + M(1 - v)
# 3x + y + 1 >= 5 - M(1 - v)
# u + v >= 1
# u,v in {0,1}

- $\sum_j a_{1j} x = b_1$ or $\sum_j a_{2j} x = b_2$ (两个约束至少一个成立)
    - 线性化方法:
        - $\sum_j a_{ij} x \le b_i + M(1 - y_i)$, $i = 1,2$
        - $\sum_j a_{ij} x \ge b_i - M(1 - y_i)$, $i = 1,2$
        - $\sum_i y_i \ge 1$
        - $y_i \in \{0,1\}$, $i = 1,2$

In [131]:
# 例子
# x + 2y + 10 = 15 or 3x + y + 1 = 5
# ->
# x + 2y + 10 <= 15 + M(1 - u)
# x + 2y + 10 >= 15 - M(1 - u)
# 3x + y + 1 <= 5 + M(1 - v)
# 3x + y + 1 >= 5 - M(1 - v)
# u + v >= 1
# u,v in {0,1}

### 乘积式

- $x_1 x_2$, 其中 $x_1, x_2 \in \{0,1\}$
    - 线性化方法: $y = x_1 x_2$
        - $y \le x_1$
        - $y \le x_2$
        - $y \ge x_1 + x_2 - 1$
        - $y \in \{0,1\}$

- $x_1 x_2$, 其中 $x_1 \in \{0,1\}$, $x_2 \in [0, u]$
    - 线性化方法: $y = x_1 x_2$
        - $y \le u x_1$
        - $y \le x_2$
        - $y \ge x_2 - u(1 - x_1)$
        - $y \in \{0,u\}$

- $x_1 x_2$, 其中 $x_1 \in \{0,1\}$, $x_2 \in [l, u]$
    - 线性化方法: $y = x_1 x_2$
        - $y \le x_2$
        - $y \ge x_2 - u(1 - x_1)$
        - $l x_1 \le y \le u x_1$

### 部分整数变量

- $z \in [0, a] \ integer$  or  $z \in [a, b] \ continuous$
    - 线性化方法: 令
    - $x \in [0,a] \ integer$
    - $y \in [a, b] \ continuous$
    - $u \in \{0,1\}$
        - $z \le x + (1 - u)M$
        - $z \ge x - (1 - u)M$
        - $z \le y + u M$
        - $z \ge y - u M$